In [62]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd, numpy as np, re, time
import itertools

import seaborn as sns
sns.set(style = "whitegrid", color_codes = True,font_scale = 1.5)

from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')

In [63]:
from collections import defaultdict
import re

import sys
import os

os.environ['KERAS_BACKEND']='theano'

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, Concatenate, Dropout
from keras.models import Sequential
from keras.models import Model

import pickle
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from collections import defaultdict

# Reddit Dataset

In [103]:
df = pd.read_csv("Reddit_Dataset_Cleaned.csv")

In [104]:
print(len(df))

1010773


In [105]:
features = df['comment']
labels = df['label']
print(len(features))
print(len(labels))

1010773
1010773


### Select desired sample size:

In [114]:
df = df.sample(28619)
df.reset_index(drop=True,inplace=True)

In [115]:
features = df['comment']
labels = df['label']
print(len(features))
print(len(labels))

28619
28619


# CNN Reddit

In [117]:
score_list = []
reddit_indices = []

In [118]:
from sklearn.model_selection import train_test_split
# from Word2VecUtility import Word2VecUtility
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from collections import defaultdict

def get_volcabulary_and_list_words(data):
    reviews_words = []
    volcabulary = defaultdict(int)
    for review in data["comment"]:
        # review_words = Word2VecUtility.review_to_wordlist(review, remove_stopwords=True)
        review_words = review.split()
        reviews_words.append(review_words)
        for word in review_words:
            volcabulary[word] += 1
    return volcabulary, reviews_words

def get_reviews_word_index(reviews_words, volcabulary, max_words, max_length):
    volcabulary = sorted(volcabulary.items(), key = lambda x : x[1], reverse = True)[:max_words]
    word2index = {word[0]: i for i, word in enumerate(volcabulary)}
    reviews_words_index = [[start] + [(word2index[w] + index_from) if w in word2index else oov for w in review] for review in reviews_words]
    # in word2vec embedding, use (i < max_words + index_from) because we need the exact index for each word, in order to map it to its vector. And then its max_words is 5003 instead of 5000.
    # padding with 0, each review has max_length now.
    reviews_words_index = sequence.pad_sequences(reviews_words_index, maxlen=max_length, padding='post', truncating='post')
    return reviews_words_index


data = df

# data processing para
max_words = 5000
max_length = 50

# model training parameters
batch_size = int(0.8*len(data))//2
embedding_dims = 100
nb_filter = 250
filter_length = 3
hidden_dims = 250
#***** nb_epoch = 2

# index trick parameters
index_from = 3
start = 1
# padding = 0
oov = 2

counter=0
best_score = 0
for i in range(1):
    nb_epoch=i

#         print('get volcabulary...')
    volcabulary, reviews_words = get_volcabulary_and_list_words(data)
#         print('get reviews_words_index...')
    reviews_words_index = get_reviews_word_index(reviews_words, volcabulary, max_words, max_length)

#         print(reviews_words_index[:20, :12])
#         print(reviews_words_index.shape)

    labels = data["label"]
    labels[labels == 0] = 0
    labels[labels == 1] = 1

    pickle.dump((reviews_words_index, labels), open("399850by50reviews_words_index.pkl", 'wb'))
    # (reviews_words_index, labels) = pickle.load(open("399850by50reviews_word2vec_words_index.pkl", 'rb'))


    index = np.arange(reviews_words_index.shape[0])
    train_index, valid_index = train_test_split(
        index, train_size=0.8, random_state=520)

    train_data = reviews_words_index[train_index]
    valid_data = reviews_words_index[valid_index]
    train_labels = labels[train_index]
    valid_labels = labels[valid_index]
#         print(train_data.shape)
#         print(valid_data.shape)

    del(labels, train_index, valid_index)

    print("start training model...")

    model = Sequential()

    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    model.add(Embedding(max_words + index_from, embedding_dims, \
                        input_length=max_length))
    model.add(Dropout(0.25))

    # we add a Convolution1D, which will learn nb_filter
    # word group filters of size filter_length:

    # filter_length is like filter size, subsample_length is like step in 2D CNN.
    model.add(Convolution1D(nb_filter=nb_filter,
                            filter_length=filter_length,
                            border_mode='valid',
                            activation='relu',
                            subsample_length=1))
    # we use standard max pooling (halving the output of the previous layer):
    model.add(MaxPooling1D(pool_length=2))

    # We flatten the output of the conv layer,
    # so that we can add a vanilla dense layer:
    model.add(Flatten())

    # We add a vanilla hidden layer:
    model.add(Dense(hidden_dims))
    model.add(Dropout(0.25))
    model.add(Activation('relu'))

    # We project onto a single unit output layer, and squash it with a sigmoid:
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop')
    model.fit(train_data, train_labels, batch_size=batch_size,
              nb_epoch=nb_epoch,
              validation_data=(valid_data, valid_labels))
    scores = model.evaluate(valid_data, valid_labels, verbose=0)
#     print(scores)
    if scores > best_score:
        best_score = scores
    score_list.append(scores)
    reddit_indices.append(counter)
    counter+=1
print('best_score:', max(score_list))
print('index of best_score:', reddit_indices)

start training model...
Train on 22895 samples, validate on 5724 samples
best_score: 0.6931684421210086
index of best_score: [0]


In [119]:
print(score_list)

[0.6931684421210086]


# "News Headline" Dataset

In [120]:
# SOURCE: https://github.com/rishabhmisra/News-Headlines-Dataset-For-Sarcasm-Detection
# NOTES - Dataset is much cleaner, less spelling errors, higher modeling scores overall
df = pd.read_json("Sarcasm_Headlines_Dataset.json", lines = True)

In [126]:
print("Number of Headlines:", len(df))

Number of Headlines: 28619


In [128]:
features = df['headline']
labels = df['is_sarcastic']
print(len(features))
print(len(labels))

28619
28619


# CNN New Headlines

In [129]:
data = pd.read_json('Sarcasm_Headlines_Dataset.json',lines=True)

In [130]:
new_score_list = []
indices = []

In [131]:
import pickle
from sklearn.model_selection import train_test_split
# from Word2VecUtility import Word2VecUtility
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from collections import defaultdict

def get_volcabulary_and_list_words(data):
    reviews_words = []
    volcabulary = defaultdict(int)
    for review in data["headline"]:
        # review_words = Word2VecUtility.review_to_wordlist(review, remove_stopwords=True)
        review_words = review.split()
        reviews_words.append(review_words)
        for word in review_words:
            volcabulary[word] += 1
    return volcabulary, reviews_words

def get_reviews_word_index(reviews_words, volcabulary, max_words, max_length):
    volcabulary = sorted(volcabulary.items(), key = lambda x : x[1], reverse = True)[:max_words]
    word2index = {word[0]: i for i, word in enumerate(volcabulary)}
    reviews_words_index = [[start] + [(word2index[w] + index_from) if w in word2index else oov for w in review] for review in reviews_words]
    # in word2vec embedding, use (i < max_words + index_from) because we need the exact index for each word, in order to map it to its vector. And then its max_words is 5003 instead of 5000.
    # padding with 0, each review has max_length now.
    reviews_words_index = sequence.pad_sequences(reviews_words_index, maxlen=max_length, padding='post', truncating='post')
    return reviews_words_index

# data processing para
max_words = 5000
max_length = 50

# model training parameters
batch_size = int(0.8*len(data))//2
embedding_dims = 100
nb_filter = 250
filter_length = 3
hidden_dims = 250
# nb_epoch = 2

# index trick parameters
index_from = 3
start = 1
# padding = 0
oov = 2

counter = 0
best_score = 0
for i in range(1,4):
    nb_epoch=i

#         print('get volcabulary...')
    volcabulary, reviews_words = get_volcabulary_and_list_words(data)
#         print('get reviews_words_index...')
    reviews_words_index = get_reviews_word_index(reviews_words, volcabulary, max_words, max_length)

#         print(reviews_words_index[:20, :12])
#         print(reviews_words_index.shape)

    labels = data["is_sarcastic"]
    labels[labels == 0] = 0
    labels[labels == 1] = 1

    pickle.dump((reviews_words_index, labels), open("399850by50reviews_words_index.pkl", 'wb'))
    # (reviews_words_index, labels) = pickle.load(open("399850by50reviews_word2vec_words_index.pkl", 'rb'))


    index = np.arange(reviews_words_index.shape[0])
    train_index, valid_index = train_test_split(
        index, train_size=0.8, random_state=520)

    train_data = reviews_words_index[train_index]
    valid_data = reviews_words_index[valid_index]
    train_labels = labels[train_index]
    valid_labels = labels[valid_index]
#         print(train_data.shape)
#         print(valid_data.shape)

    del(labels, train_index, valid_index)

    print("start training model...")

    model = Sequential()

    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    model.add(Embedding(max_words + index_from, embedding_dims, \
                        input_length=max_length))
    model.add(Dropout(0.25))

    # we add a Convolution1D, which will learn nb_filter
    # word group filters of size filter_length:

    # filter_length is like filter size, subsample_length is like step in 2D CNN.
    model.add(Convolution1D(nb_filter=nb_filter,
                            filter_length=filter_length,
                            border_mode='valid',
                            activation='relu',
                            subsample_length=1))
    # we use standard max pooling (halving the output of the previous layer):
    model.add(MaxPooling1D(pool_length=2))

    # We flatten the output of the conv layer,
    # so that we can add a vanilla dense layer:
    model.add(Flatten())

    # We add a vanilla hidden layer:
    model.add(Dense(hidden_dims))
    model.add(Dropout(0.25))
    model.add(Activation('relu'))

    # We project onto a single unit output layer, and squash it with a sigmoid:
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop')
    model.fit(train_data, train_labels, batch_size=batch_size,
              nb_epoch=nb_epoch,
              validation_data=(valid_data, valid_labels))
    scores = model.evaluate(valid_data, valid_labels, verbose=0)
    if scores > best_score:
        best_score = scores
    new_score_list.append(scores)
    indices.append(counter)
    counter+=1
print('best_score:', max(new_score_list))
print('index of best_score:', indices)

start training model...
Train on 22895 samples, validate on 5724 samples
Epoch 1/1
22895/22895 [==============================] - 49s 2ms/step - loss: 0.9326 - val_loss: 0.6889
start training model...
Train on 22895 samples, validate on 5724 samples
Epoch 1/2
22895/22895 [==============================] - 51s 2ms/step - loss: 0.8542 - val_loss: 0.7553
Epoch 2/2
22895/22895 [==============================] - 56s 2ms/step - loss: 0.7546 - val_loss: 0.7227
start training model...
Train on 22895 samples, validate on 5724 samples
Epoch 1/3
22895/22895 [==============================] - 49s 2ms/step - loss: 1.0692 - val_loss: 0.7100
Epoch 2/3
22895/22895 [==============================] - 60s 3ms/step - loss: 0.7138 - val_loss: 0.7077
Epoch 3/3
22895/22895 [==============================] - 43s 2ms/step - loss: 0.7137 - val_loss: 0.7083
best_score: 0.7226558415978043
index of best_score: [0, 1, 2]


In [132]:
new_score_list

[0.6889123063167436, 0.7226558415978043, 0.7083130971190814]